In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

train_data_path = "./train/"
val_data_path = "./val/"
test_data_path = "./test/"


transform_pipeline = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform_pipeline)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform_pipeline)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform_pipeline)

# Data loaders
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

# Model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.dropout1=nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(84, 50)
        self.dropout2=nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):  # flatten
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x=self.dropout1(x)
        x = F.relu(self.fc2(x))
        x=self.dropout1(x)
        x = self.fc3(x)
        return x

# Initialization
model = SimpleNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
    model.to(device)

    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss_value = loss_fn(outputs, targets)  
            loss_value.backward() #calculate gradients
            optimizer.step() #update weights
            training_loss += loss_value.data.item()
        training_loss /= len(train_loader)

        model.eval()
        num_correct = 0
        num_examples = 0

        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss_value = loss_fn(outputs, targets)
            valid_loss += loss_value.data.item()
            correct = torch.eq(torch.max(F.softmax(outputs, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]

        valid_loss /= len(val_loader)
        accuracy = num_correct / num_examples * 100

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, Accuracy: {:.2f}%'.format(
            epoch + 1, training_loss, valid_loss, accuracy))

# Run training
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader)





Epoch: 1, Training Loss: 4.12, Validation Loss: 2.70, Accuracy: 44.44%
Epoch: 2, Training Loss: 2.79, Validation Loss: 1.15, Accuracy: 69.44%
Epoch: 3, Training Loss: 1.48, Validation Loss: 1.42, Accuracy: 46.30%
Epoch: 4, Training Loss: 1.43, Validation Loss: 0.67, Accuracy: 68.52%
Epoch: 5, Training Loss: 0.91, Validation Loss: 0.59, Accuracy: 69.44%
Epoch: 6, Training Loss: 0.83, Validation Loss: 0.55, Accuracy: 73.15%
Epoch: 7, Training Loss: 0.73, Validation Loss: 0.59, Accuracy: 72.22%
Epoch: 8, Training Loss: 0.67, Validation Loss: 0.57, Accuracy: 75.00%
Epoch: 9, Training Loss: 0.59, Validation Loss: 0.58, Accuracy: 73.15%
Epoch: 10, Training Loss: 0.55, Validation Loss: 0.58, Accuracy: 72.22%
Epoch: 11, Training Loss: 0.60, Validation Loss: 0.53, Accuracy: 74.07%
Epoch: 12, Training Loss: 0.54, Validation Loss: 0.54, Accuracy: 78.70%
Epoch: 13, Training Loss: 0.54, Validation Loss: 0.54, Accuracy: 79.63%
Epoch: 14, Training Loss: 0.52, Validation Loss: 0.54, Accuracy: 80.56%
E

In [ ]:
#Test

from PIL import Image
labels = ['cat','fish']
img = Image.open('./val/fish/HPIM1121.JPG')
img = transform_pipeline(img)
img = img.unsqueeze(0)
prediction = model(img)
prediction = prediction.argmax()
print(labels[prediction])

In [ ]:
#saving model
torch.save(model.state_dict(), "./Model/model_weights.pth")
